In [1]:
import os
import config
import platform
v1, v2, v3 = config.spark_env(platform.node())
os.environ['SPARK_VERSION'] = v1
os.environ['JAVA_HOME'] = v2
os.environ['SPARK_HOME'] = v3
import findspark
import pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("emissionsdataframe").getOrCreate()
from sqlalchemy import create_engine, insert
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import MetaData, update, Table
from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
import pandas as pd
from pyspark.sql.types import IntegerType,BooleanType,DateType
from pyspark.sql.functions import col
from pyspark.sql import Row
from pyspark.sql.functions import array, col, explode, lit, struct, log
from pyspark.sql import DataFrame
from typing import Iterable
import numpy as np
import spark_functions
import tensorflow as tf
import keras.metrics
import pyspark.sql.functions as F
import math
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pystan
import fbprophet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from fbprophet import Prophet
from sklearn.model_selection import train_test_split
import scipy.stats as st
rds_string = config.rds_string
engine = create_engine(f'postgresql://{rds_string}')
conn = engine.connect()
metadata = MetaData(engine)

In [2]:
countries = ["India", "China", "United States"]

In [3]:
def df_maker(arr):
    c_dict = {}
    for i in arr:
        temp_dict = {}
        df, cols = spark_functions.merged_df(i)
        c_dict[i] = {'df': df}
        c_dict['cols'] = cols
    steve = len(arr)    
    df = c_dict[arr[0]]['df']
    for i in range(steve - 1):
        column = arr[i+1]
        df2 = c_dict[column]['df']
        df = df.union(df2)
    c_dict['union'] = {'df':df}
    return c_dict

diction = df_maker(countries)

def make_models(diction):
    col_len = len(diction['cols'][2:])
    dict_ = {}
    for i in range(col_len):
        tempz_dict = {}
        temp_dict = {}
        colz = diction['cols'][2:]
        y = colz[i]
        colz.remove(colz[i])
        x = colz

        arr = np.array(diction['union']['df'].select(
         x[0],
         x[1],
         x[2],
         x[3],
         x[4],
         x[5],
         x[6]) \
        .collect(),dtype = 'float')    
        arr2 = np.array(diction['union']['df'].select(y).collect(),dtype = 'float')
        tempz_dict = {'x':arr,'y':arr2}
        X_train, X_test, y_train, y_test = train_test_split(arr, arr2, test_size=0.2)
        X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.2)
        tempz_dict['train'] = [X_train,y_train]
        tempz_dict['test'] = [X_test,y_test]
        tempz_dict['val'] = [X_val,y_val]
        X_scaler = StandardScaler().fit(X_train)
        tempz_dict['scaler'] = X_scaler
        X_train_scaled = X_scaler.transform(X_train)
        X_test_scaled = X_scaler.transform(X_test)
        X_val_scaled = X_scaler.transform(X_val)
        model = Sequential()
        model.add(Dense(units=100, activation='relu', input_dim=7))
        model.add(Dense(units=100, activation='relu'))
        model.add(Dense(units=100, activation='ReLU'))
        model.add(Dense(units=100, activation='PReLU'))
        model.add(Dense(units=100, activation='swish'))
        model.add(Dense(units=1, activation='linear'))
        METRICS = [
              keras.metrics.MeanAbsoluteError(name='mean_absolute_error'),
              keras.metrics.MeanSquaredError(name='mean_squared_error')
        ]


        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='mse', 
            verbose=1,
            patience=10,
            mode='max',
            restore_best_weights=True)

        EPOCHS = 100    
        model.compile(optimizer='adam',
                  loss='mse',
                  metrics=METRICS)

        model_history = model.fit(
        X_train_scaled,
        y_train,
        epochs=EPOCHS,
        validation_data=(X_test_scaled, y_test)
        )
        tempz_dict['model'] = model
        diction[y] = tempz_dict
    return diction

diction = make_models(diction)

def spark_to_prophet(df, column):
    arr = np.array(df.collect())
    df = pd.DataFrame(arr, columns = column)
    ds_len = len(df)
    dates = []
    for i in range(ds_len):
        year = str(int(df['year'][i]))
        year = year + "-01-01"
        dates.append(year)
    df['year'] = dates
    return df

def make_prophet_model(df,periods):
    m = Prophet()
    m.fit(df)
    future = m.make_future_dataframe(periods=periods, freq='Y')
    forecast = m.predict(future)
    return (forecast, m)



Epoch 1/100
2/2 [==============================] - 1s 137ms/step - loss: 700608896.0000 - mean_absolute_error: 18767.1074 - mean_squared_error: 700608896.0000 - val_loss: 357504256.0000 - val_mean_absolute_error: 10210.8906 - val_mean_squared_error: 357504256.0000
Epoch 2/100
2/2 [==============================] - 0s 20ms/step - loss: 700602688.0000 - mean_absolute_error: 18766.8984 - mean_squared_error: 700602688.0000 - val_loss: 357499360.0000 - val_mean_absolute_error: 10210.5244 - val_mean_squared_error: 357499360.0000
Epoch 3/100
2/2 [==============================] - 0s 20ms/step - loss: 700593984.0000 - mean_absolute_error: 18766.6133 - mean_squared_error: 700593984.0000 - val_loss: 357492512.0000 - val_mean_absolute_error: 10210.0039 - val_mean_squared_error: 357492512.0000
Epoch 4/100
2/2 [==============================] - 0s 20ms/step - loss: 700581696.0000 - mean_absolute_error: 18766.2090 - mean_squared_error: 700581696.0000 - val_loss: 357482336.0000 - val_mean_absolute_er

Epoch 32/100
2/2 [==============================] - 0s 22ms/step - loss: 510464384.0000 - mean_absolute_error: 15331.2178 - mean_squared_error: 510464384.0000 - val_loss: 247423824.0000 - val_mean_absolute_error: 9046.2959 - val_mean_squared_error: 247423824.0000
Epoch 33/100
2/2 [==============================] - 0s 20ms/step - loss: 468491008.0000 - mean_absolute_error: 14646.0723 - mean_squared_error: 468491040.0000 - val_loss: 227111472.0000 - val_mean_absolute_error: 9080.7617 - val_mean_squared_error: 227111472.0000
Epoch 34/100
2/2 [==============================] - 0s 21ms/step - loss: 422572256.0000 - mean_absolute_error: 13819.9258 - mean_squared_error: 422572256.0000 - val_loss: 205103760.0000 - val_mean_absolute_error: 9064.3604 - val_mean_squared_error: 205103760.0000
Epoch 35/100
2/2 [==============================] - 0s 21ms/step - loss: 366152736.0000 - mean_absolute_error: 12892.8008 - mean_squared_error: 366152736.0000 - val_loss: 182377296.0000 - val_mean_absolute_er

2/2 [==============================] - 0s 21ms/step - loss: 12870459.0000 - mean_absolute_error: 2156.8750 - mean_squared_error: 12870459.0000 - val_loss: 5690578.5000 - val_mean_absolute_error: 2095.7249 - val_mean_squared_error: 5690578.5000
Epoch 64/100
2/2 [==============================] - 0s 19ms/step - loss: 12477332.0000 - mean_absolute_error: 2013.4652 - mean_squared_error: 12477332.0000 - val_loss: 4969384.5000 - val_mean_absolute_error: 1940.1320 - val_mean_squared_error: 4969384.5000
Epoch 65/100
2/2 [==============================] - 0s 20ms/step - loss: 12383147.0000 - mean_absolute_error: 1929.8324 - mean_squared_error: 12383147.0000 - val_loss: 4457957.5000 - val_mean_absolute_error: 1813.6693 - val_mean_squared_error: 4457957.5000
Epoch 66/100
2/2 [==============================] - 0s 19ms/step - loss: 12157310.0000 - mean_absolute_error: 1892.5653 - mean_squared_error: 12157310.0000 - val_loss: 4178218.7500 - val_mean_absolute_error: 1749.1058 - val_mean_squared_error

2/2 [==============================] - 0s 19ms/step - loss: 8601606.0000 - mean_absolute_error: 1670.6356 - mean_squared_error: 8601606.0000 - val_loss: 4414012.0000 - val_mean_absolute_error: 1675.0746 - val_mean_squared_error: 4414012.0000
Epoch 96/100
2/2 [==============================] - 0s 20ms/step - loss: 8520181.0000 - mean_absolute_error: 1667.2975 - mean_squared_error: 8520181.0000 - val_loss: 4460623.5000 - val_mean_absolute_error: 1671.5677 - val_mean_squared_error: 4460623.5000
Epoch 97/100
2/2 [==============================] - 0s 19ms/step - loss: 8451187.0000 - mean_absolute_error: 1660.8575 - mean_squared_error: 8451187.0000 - val_loss: 4471204.5000 - val_mean_absolute_error: 1659.9268 - val_mean_squared_error: 4471204.5000
Epoch 98/100
2/2 [==============================] - 0s 20ms/step - loss: 8389607.0000 - mean_absolute_error: 1657.1274 - mean_squared_error: 8389607.0000 - val_loss: 4509524.0000 - val_mean_absolute_error: 1656.3383 - val_mean_squared_error: 450952

2/2 [==============================] - 0s 19ms/step - loss: 1.9212 - mean_absolute_error: 1.0148 - mean_squared_error: 1.9212 - val_loss: 0.8607 - val_mean_absolute_error: 0.7945 - val_mean_squared_error: 0.8607
Epoch 31/100
2/2 [==============================] - 0s 20ms/step - loss: 1.7409 - mean_absolute_error: 0.9631 - mean_squared_error: 1.7409 - val_loss: 0.9550 - val_mean_absolute_error: 0.8001 - val_mean_squared_error: 0.9550
Epoch 32/100
2/2 [==============================] - 0s 20ms/step - loss: 1.6910 - mean_absolute_error: 0.9525 - mean_squared_error: 1.6910 - val_loss: 1.0331 - val_mean_absolute_error: 0.8132 - val_mean_squared_error: 1.0331
Epoch 33/100
2/2 [==============================] - 0s 20ms/step - loss: 1.5805 - mean_absolute_error: 0.9144 - mean_squared_error: 1.5805 - val_loss: 0.9663 - val_mean_absolute_error: 0.7580 - val_mean_squared_error: 0.9663
Epoch 34/100
2/2 [==============================] - 0s 20ms/step - loss: 1.4836 - mean_absolute_error: 0.8654 - m

...

2/2 [==============================] - 0s 19ms/step - loss: 0.9762 - mean_absolute_error: 0.7601 - mean_squared_error: 0.9762 - val_loss: 15.8300 - val_mean_absolute_error: 1.6019 - val_mean_squared_error: 15.8300
Epoch 35/100
2/2 [==============================] - 0s 20ms/step - loss: 0.8779 - mean_absolute_error: 0.7076 - mean_squared_error: 0.8779 - val_loss: 14.6803 - val_mean_absolute_error: 1.5521 - val_mean_squared_error: 14.6803
Epoch 36/100
2/2 [==============================] - 0s 20ms/step - loss: 0.7886 - mean_absolute_error: 0.6638 - mean_squared_error: 0.7886 - val_loss: 13.7233 - val_mean_absolute_error: 1.5393 - val_mean_squared_error: 13.7233
Epoch 37/100
2/2 [==============================] - 0s 20ms/step - loss: 0.7804 - mean_absolute_error: 0.6606 - mean_squared_error: 0.7804 - val_loss: 13.1936 - val_mean_absolute_error: 1.5273 - val_mean_squared_error: 13.1936
Epoch 38/100
2/2 [==============================] - 0s 19ms/step - loss: 0.7854 - mean_absolute_error: 0.

Epoch 70/100
2/2 [==============================] - 0s 19ms/step - loss: 0.2249 - mean_absolute_error: 0.3647 - mean_squared_error: 0.2249 - val_loss: 11.1715 - val_mean_absolute_error: 1.4100 - val_mean_squared_error: 11.1715
Epoch 71/100
2/2 [==============================] - 0s 23ms/step - loss: 0.2174 - mean_absolute_error: 0.3583 - mean_squared_error: 0.2174 - val_loss: 11.1360 - val_mean_absolute_error: 1.4114 - val_mean_squared_error: 11.1360
Epoch 72/100
2/2 [==============================] - 0s 19ms/step - loss: 0.2096 - mean_absolute_error: 0.3550 - mean_squared_error: 0.2096 - val_loss: 11.0304 - val_mean_absolute_error: 1.4089 - val_mean_squared_error: 11.0304
Epoch 73/100
2/2 [==============================] - 0s 19ms/step - loss: 0.2002 - mean_absolute_error: 0.3466 - mean_squared_error: 0.2002 - val_loss: 10.8922 - val_mean_absolute_error: 1.3978 - val_mean_squared_error: 10.8922
Epoch 74/100
2/2 [==============================] - 0s 19ms/step - loss: 0.1917 - mean_absol

2/2 [==============================] - 0s 20ms/step - loss: 85.9525 - mean_absolute_error: 9.2128 - mean_squared_error: 85.9525 - val_loss: 78.0253 - val_mean_absolute_error: 8.7126 - val_mean_squared_error: 78.0253
Epoch 7/100
2/2 [==============================] - 0s 20ms/step - loss: 68.7097 - mean_absolute_error: 8.1807 - mean_squared_error: 68.7097 - val_loss: 59.3813 - val_mean_absolute_error: 7.4594 - val_mean_squared_error: 59.3813
Epoch 8/100
2/2 [==============================] - 0s 19ms/step - loss: 48.4707 - mean_absolute_error: 6.7328 - mean_squared_error: 48.4707 - val_loss: 39.5514 - val_mean_absolute_error: 5.7294 - val_mean_squared_error: 39.5514
Epoch 9/100
2/2 [==============================] - 0s 19ms/step - loss: 28.1101 - mean_absolute_error: 4.8208 - mean_squared_error: 28.1101 - val_loss: 23.4079 - val_mean_absolute_error: 4.5800 - val_mean_squared_error: 23.4079
Epoch 10/100
2/2 [==============================] - 0s 19ms/step - loss: 14.4762 - mean_absolute_err

2/2 [==============================] - 0s 22ms/step - loss: 1.3290 - mean_absolute_error: 0.8749 - mean_squared_error: 1.3290 - val_loss: 0.8790 - val_mean_absolute_error: 0.8271 - val_mean_squared_error: 0.8790
Epoch 43/100
2/2 [==============================] - 0s 22ms/step - loss: 1.2603 - mean_absolute_error: 0.8552 - mean_squared_error: 1.2603 - val_loss: 0.8102 - val_mean_absolute_error: 0.7937 - val_mean_squared_error: 0.8102
Epoch 44/100
2/2 [==============================] - 0s 21ms/step - loss: 1.1652 - mean_absolute_error: 0.8201 - mean_squared_error: 1.1652 - val_loss: 0.7179 - val_mean_absolute_error: 0.7185 - val_mean_squared_error: 0.7179
Epoch 45/100
2/2 [==============================] - 0s 21ms/step - loss: 1.0539 - mean_absolute_error: 0.7728 - mean_squared_error: 1.0539 - val_loss: 0.6593 - val_mean_absolute_error: 0.6516 - val_mean_squared_error: 0.6593
Epoch 46/100
2/2 [==============================] - 0s 20ms/step - loss: 0.9729 - mean_absolute_error: 0.7367 - m

2/2 [==============================] - 0s 20ms/step - loss: 0.0825 - mean_absolute_error: 0.2059 - mean_squared_error: 0.0825 - val_loss: 0.1116 - val_mean_absolute_error: 0.2435 - val_mean_squared_error: 0.1116
Epoch 79/100
2/2 [==============================] - 0s 19ms/step - loss: 0.0810 - mean_absolute_error: 0.2055 - mean_squared_error: 0.0810 - val_loss: 0.1139 - val_mean_absolute_error: 0.2441 - val_mean_squared_error: 0.1139
Epoch 80/100
2/2 [==============================] - 0s 19ms/step - loss: 0.0779 - mean_absolute_error: 0.2027 - mean_squared_error: 0.0779 - val_loss: 0.1129 - val_mean_absolute_error: 0.2412 - val_mean_squared_error: 0.1129
Epoch 81/100
2/2 [==============================] - 0s 20ms/step - loss: 0.0773 - mean_absolute_error: 0.2011 - mean_squared_error: 0.0773 - val_loss: 0.1116 - val_mean_absolute_error: 0.2376 - val_mean_squared_error: 0.1116
Epoch 82/100
2/2 [==============================] - 0s 21ms/step - loss: 0.0778 - mean_absolute_error: 0.2024 - m

In [4]:
def proph(diction,periods):
    dict__= {}
    
    for i in countries:
        dict_ = {}
        df = diction[i]['df']
        cols = df.columns
        df = spark_to_prophet(df,cols)
        test_cols = cols[2:]
        yhat_df = pd.DataFrame()
        yhat_lower_df = pd.DataFrame()
        yhat_upper_df = pd.DataFrame()
        country_l = []
        for j in test_cols:
            future_df = pd.DataFrame()
            future_df['ds'] = df['year']
            future_df['y'] = df[j]
            df2, m = make_prophet_model(future_df,periods)
            dict_[j] = {'model':m,'future':df2}
        dict__[i] = dict_
    diction['prophet'] = dict__
    return diction

diction = proph(diction,10)



INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 20.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 20.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 20.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run

In [77]:
def hats(diction):
    dictionary = {}

    for c in countries:
        rows = diction[c]['df'].count()
        columns = diction['cols'][2:]
        lenz = len(columns)
        df_y = pd.DataFrame()
        df_low = pd.DataFrame()
        df_up = pd.DataFrame()
        c_list = []
        for l in columns:

            try:
                df_y['country']
            except:
                holdz = len(diction['prophet'][c][l]['future'])
                fin_hold = holdz - rows
                for r in range(holdz):
                    c_list.append(c)                
                df_y['country'] =c_list
                df_y['year'] = diction['prophet'][c][l]['future']['ds']
                df_low['country'] =c_list
                df_low['year'] = diction['prophet'][c][l]['future']['ds']
                df_up['country'] =c_list
                df_up['year'] = diction['prophet'][c][l]['future']['ds']
            df_y[l] = diction['prophet'][c][l]['future']['yhat']
            df_low[l] = diction['prophet'][c][l]['future']['yhat']
            df_up[l] = diction['prophet'][c][l]['future']['yhat']

        dictionary[c] = {'yhat':df_y, 'yhat_lower':df_low, 'yhat_upper':df_up}
    diction['prophet_predictions'] = dictionary       
    return diction

diction = hats(diction)

In [84]:
def dl_preds(diction):    
    lenz = diction['India']['df'].count()
    dictionary = {}
    for c in countries:

        model_yhat_df = pd.DataFrame()
        model_yhat_lower_df = pd.DataFrame()
        model_yhat_upper_df = pd.DataFrame()
        yhat = diction['prophet_predictions'][c]['yhat'][lenz:]
        yhat_lower = diction['prophet_predictions'][c]['yhat_lower'][lenz:]
        yhat_upper = diction['prophet_predictions'][c]['yhat_upper'][lenz:]
        model_yhat_df['country'] = yhat['country']
        model_yhat_df['year'] = yhat['year']
        model_yhat_lower_df['country'] = yhat_lower['country']
        model_yhat_lower_df['year'] = yhat_lower['year']
        model_yhat_upper_df['country'] = yhat_upper['country']
        model_yhat_upper_df['year'] = yhat_upper['year']
        diction['prophet_predictions'][c]
        columns = diction['cols'][2:]
        for i in columns:
            col = diction['cols'][2:]
            X_scaler =  diction[i]['scaler']
            model = diction[i]['model']
            col.remove(i)
            x_yhat = yhat[col]
            x_yhat_lower = yhat_lower[col]
            x_yhat_upper = yhat_upper[col]

            x_yhat_norm = X_scaler.transform(x_yhat)
            x_yhat_lower_norm = X_scaler.transform(x_yhat_lower)
            x_yhat_upper_norm = X_scaler.transform(x_yhat_upper)

            model_yhat_df[i] = model.predict(x_yhat_norm)
            model_yhat_lower_df[i] = model.predict(x_yhat_lower_norm)
            model_yhat_upper_df[i] = model.predict(x_yhat_upper_norm)
        dictionary[c] = {'dl_yhat':model_yhat_df, 'dl_yhat_lower':model_yhat_lower_df, 'dl_yhat_upper':model_yhat_upper_df}
    diction['dl_prediction_dfs'] = dictionary
    return diction

In [86]:
diction = dl_preds(diction)

In [91]:
diction['dl_prediction_dfs']['India']['dl_yhat']

,country,year,gdp,annual_co2_emissions_tonnes_log,import_trade_sum_usd_log,import_weight_sum_kg_log,import_quantity_sum_log,export_trade_sum_usd_log,export_weight_sum_kg_log,export_quantity_sum_log
27,India,2016-12-31,10939.265625,9.580269,11.874404,11.615630,11.581485,10.631338,10.104944,10.393364
28,India,2017-12-31,15650.452148,9.983238,11.715800,11.501826,11.345828,9.393957,8.878062,10.477627
29,India,2018-12-31,24448.144531,10.947771,12.402023,11.822759,11.460323,8.970545,8.637923,11.536982
30,India,2019-12-31,37405.160156,11.894262,14.126561,13.215495,12.827147,11.204233,10.310929,13.941467
31,India,2020-12-31,14818.939453,10.046816,12.656904,12.114771,11.814901,10.893660,10.940834,10.472267
32,India,2021-12-31,19312.453125,10.337530,12.822233,12.002115,11.797078,9.735474,9.838751,10.705119
33,India,2022-12-31,27342.917969,11.119841,13.541109,12.520918,12.065998,9.607622,9.355635,11.628179
34,India,2023-12-31,39869.464844,12.061424,15.264333,13.678535,13.226468,11.576320,10.407610,14.299841
35,India,2024-12-31,18820.082031,10.668222,13.864056,13.077945,12.707350,11.500597,12.287914,11.026383
36,India,2025-12-31,23147.812500,10.897137,14.166652,12.964791,12.646888,10.496202,11.277649,11.370621


In [ ]:

def comp_plts(column):
    helds = []
    for (key,value) in dict_column.items():
        helds.append(value)
    x_axis = helds[0]
    y_axis = helds[1]
    return (x_axis, y_axis)

In [ ]:
def predictions(cols,model,dictionary):
    lenny = len(dictionary['dfs'])
    hold2 = []
    for i in range(lenny):
        hold = dictionary['dfs'][i]
        for (key, value) in hold.items():
            hold2.append(value)
    dictionary2 = {}
    for i in range(3):
        temp_dict ={}
        df1 = hold2[3]
        df2 = hold2[i]
        filt_len = len(df1)
        df3 = hold2[i][filt_len:]
        holder = cols[2:3] + cols[4:]
        future_features = df3[holder]
        future_feat_norm = X_scaler.transform(future_features)
        t1 = model.predict(future_feat_norm)
        t2 = df3['annual_co2_emissions_tonnes_log'].reset_index(drop=True)
        t_len = len(t1)
        predi = []
        actu = []
        for j in range(t_len):
            predi.append(t1[j][0].astype(float))
            actu.append(t2[j].astype(float))
        temp_dict['dl_predictions'] = predi
        temp_dict['prophet_pred'] = actu
        if i == 0:
             dictionary2['yhat'] = temp_dict
        elif i == 1:
            dictionary2['yhat_lower'] = temp_dict
        elif i == 2:
            dictionary2['yhat_upper'] = temp_dict
            
        
    return (dictionary2)

def comp_plts(dict_column):
    helds = []
    for (key,value) in dict_column.items():
        helds.append(value)
    x_axis = helds[0]
    y_axis = helds[1]
    return (x_axis, y_axis)

In [ ]:
comp_plts(dict_test['yhat'])

In [ ]:
x_axis, y_axis = comp_plts(dict_test['yhat'])
plt.scatter(x_axis, y_axis)

In [ ]:
dict_test = predictions(us_cols,model,test_dict)

In [ ]:
axis_(dict_test)

In [ ]:
dict_test

In [ ]:
def predictions(cols,model,dictionary):
    lenny = len(dictionary['dfs'])
    hold2 = []
    dictionary2 = {}
    for i in range(lenny):
        hold = dictionary['dfs'][i]
        for (key, value) in hold.items():
            hold2.append(value)
    
    for i in range(3):
        temp_dict ={}
        df1 = hold2[3]
        df2 = hold2[i]
        filt_len = len(df1)
        df3 = hold2[i][filt_len:]
        holder = cols[2:3] + cols[4:]
        future_features = df3[holder]
        future_feat_norm = X_scaler.transform(future_features)
        t1 = model.predict(future_feat_norm)
        t2 = df3['annual_co2_emissions_tonnes_log'].reset_index(drop=True)
        t_len = len(t1)
        predi = []
        actu = []
        for j in range(t_len):
            predi.append(t1[j][0].astype(float))
#             print(t2[0])
            actu.append(t2[j].astype(float))
        temp_dict['dl_predictions'] = predi
        temp_dict['prophet_pred'] = actu
#         print(i)
        if i == 0:
             dictionary2['yhat'] = temp_dict
        elif i == 1:
            dictionary2['yhat_lower'] = temp_dict
        elif i == 2:
            dictionary2['yhat_upper'] = temp_dict
    return

In [ ]:
def predictions(cols,model,dictionary):
    lenny = len(dictionary['dfs'])
    hold2 = []
    dictionary2 = {}
    for i in range(lenny):
        hold = dictionary['dfs'][i]
        for (key, value) in hold.items():
            hold2.append(value)
    
    for i in range(3):
        temp_dict ={}
        df1 = hold2[3]
        df2 = hold2[i]
        filt_len = len(df1)
        df3 = hold2[i][filt_len:]
        holder = cols[2:3] + cols[4:]
        future_features = df3[holder]
        future_feat_norm = X_scaler.transform(future_features)
        t1 = model.predict(future_feat_norm)
        t2 = df3['annual_co2_emissions_tonnes_log'].reset_index(drop=True)
        t_len = len(t1)
        predi = []
        actu = []
        for j in range(t_len):
            predi.append(t1[j][0].astype(float))
#             print(t2[0])
            actu.append(t2[j].astype(float))
        temp_dict['dl_predictions'] = predi
        temp_dict['prophet_pred'] = actu
#         print(i)
        if i == 0:
             dictionary2['yhat'] = temp_dict
        elif i == 1:
            dictionary2['yhat_lower'] = temp_dict
        elif i == 2:
            dictionary2['yhat_upper'] = temp_dict
    sam = []
    x_axis = []
    y_axis = []
#     print(dictionary2)
    for (key,value) in dictionary2.items():
#         print(value)
        for (keys,vals) in value.items():
            sam.append(vals)
#             print(vals)
        for z in sam:
            print(z)
#             x_axis.append(sam[i][0])
#             y_axis.append(sam[i][1])
    return (x_axis,y_axis,dictionary2)

In [ ]:
x_axis,y_axis,dict_test = predictions(us_cols,model,test_dict)

In [ ]:
x_axis

In [ ]:

sure = predictions(us_cols,model,test_dict)

In [ ]:
sure['yhat'].keys()

In [ ]:
def 
sam = []
for (key,value) in sure.items():
    for (keys,vals) in value.items():
        sam.append(vals)
axis_x = []
axis_y = []
for i in sam:
    t_len = len(sam[0])
    predi = []
    actu = []
    for i in range(t_len):
        predi.append(pred[i][0].astype(float))
        actu.append(act[i][0].astype(float))
    x = predi
    y = actu
    axis_x.append(x)
    axis_y.append(y)

In [ ]:
t_len = len(pred)
predi = []
actu = []
for i in range(t_len):
    predi.append(pred[i][0].astype(float))
    actu.append(act[i][0].astype(float))
    

In [ ]:
filt_len = len(us_pd_df)

In [ ]:
yhat_filt_df = yhat_df[filt_len:]

In [ ]:
holder = us_cols[2:3] + us_cols[4:]

In [ ]:
future_features = yhat_filt_df[holder]

In [ ]:
future_feat_norm = X_scaler.transform(future_features)

In [ ]:
future_feat_norm = X_scaler.transform(future_features)

In [ ]:
dl_pred = model.predict(future_feat_norm)

In [ ]:
prophet_pred = yhat_filt_df['annual_co2_emissions_tonnes_log']

In [ ]:
plt.scatter(dl_pred, prophet_pred)
plt.xlim([9, 13])
plt.ylim([9, 11])

In [ ]:
e_m = Prophet()
e_m.fit(us_prophet_df_f)
future = e_m.make_future_dataframe(periods=10, freq='Y')
forecast = e_m.predict(future)
fig1 = e_m.plot(forecast)

In [ ]:
forecast

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]


In [ ]:
plt.plot(forecast['ds'],forecast['yhat'])
plt.plot(forecast['ds'],forecast['yhat_upper'])
plt.plot(forecast['ds'],forecast['yhat_lower'])
plt.plot(forecast['ds'],forecast['trend'])